# Structured Publication of Life Cycle Assessment Models
## Example using Ecoinvent
### Brandon Kuczenski

Last update 3/31/2017

Please see the other notebook for more documentary information. 

Note that the ecoinvent database is subject to licensing restrictions and cannot be distributed as part of the `lca-tools-datafiles` repository.  Therefore, this example will not work unless the user has access to the Ecoinvent data files.

In [ ]:
import sys
import os

GITHUB_PATH = '/data/GitHub/'  # customize for your environment
PUBLISH_PATH = '/data/GitHub/2017/Publication-JIE/examples'

sys.path.append(os.path.join(GITHUB_PATH, 'lca-tools'))
sys.path.append(os.path.join(GITHUB_PATH, 'lca-matrix'))

import lcatools
import lcamatrix

In [ ]:
CATALOG = os.path.join(GITHUB_PATH, 'lca-tools-datafiles/catalogs/')

EI32_A = 'ecoinvent_3.2_apos_spold.json.gz' 
ELCD_LCIA = 'elcd_lcia_all.json.gz'
EI_LCIA = 'ei_lcia.json.gz'

### Flow Database

In [ ]:
LCIA = lcatools.archive_from_json(os.path.join(CATALOG, ELCD_LCIA))
EI_LCIA = lcatools.archive_from_json(os.path.join(CATALOG, EI_LCIA))
from lcatools.flowdb.flowdb import FlowDB
db = FlowDB()
_ = db.import_archive_cfs(LCIA)
_ = db.import_archive_cfs(EI_LCIA)

### LCI Background Database

In [ ]:
if os.path.exists(os.path.join(CATALOG, EI32_A)):
    # this takes about 13 seconds
    EA = lcatools.archive_from_json(os.path.join(CATALOG, EI32_A))
else:
    from lcatools.providers.ecospold2 import EcospoldV2Archive
    EA = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.zip', prefix='datasets')
    # this takes about 2 minutes:
    EA.load_all()
    # Once you have loaded the data from the ecoinvent source datafiles, save it in JSON format for easy retrieval.
    EA.write_to_file(os.path.join(CATALOG, EI32_A), exchanges=True, values=True, characterizations=True, gzip=True)

In [ ]:
from lcamatrix.background import BackgroundManager
B = BackgroundManager(EA)
B.add_all_ref_products()  # this takes about 15 seconds

In [ ]:
outputs = [f for f in B.product_flows()]
len(outputs) == 959  # like my unit test?

### Paper Example

In [ ]:
from lcamatrix.foreground import ForegroundFragment
from lcamatrix.foreground_table import ForegroundTeX
from lcamatrix.foreground_publication import ForegroundPublication

In [ ]:
my_qs = EI_LCIA.search(entity_type='quantity', Category=('eutrophication', 'marine'), Method='Midpoint')
my_qs.extend(LCIA.search(entity_type='quantity', Name=('eutrophication', 'marine')))
[str(q) for q in my_qs]

In [ ]:
pf = next(B.product_flows('potato'))
f = ForegroundFragment(B, db, pf)
for q in my_qs:
    f.characterize(q)


In [ ]:
# TeX table
t = ForegroundTeX(f, max_cols=9)
with open(os.path.join(PUBLISH_PATH, 'potato-organic_doco.tex'), 'w') as fp:
    fp.write(t.foreground_table(aggregate=True))

In [ ]:
# Excel spreadsheet
pub = ForegroundPublication(f, audit_cf=True)
pub.publish(os.path.join(PUBLISH_PATH, 'potato-organic_doco.xls'), full=True)